In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import numpy as np
from PIL import Image
from datasets import load_dataset
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import SparkSession
import io

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
sc._conf.getAll()

[('spark.sql.warehouse.dir',
  'file:/C:/Users/marco/Desktop/BDC_project/BDC_project/spark-warehouse'),
 ('spark.app.name', 'PySparkTutorial'),
 ('spark.driver.host', 'LAPTOP-JLLVBEPM'),
 ('spark.driver.memory', '45G'),
 ('spark.app.submitTime', '1683376544846'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.startTime', '1683376545165'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.

In [8]:
#open csv file
path = "C:/Users/marco/Desktop"
df_appearances = spark.read.csv(path+"/archive/appearances.csv", header=True, inferSchema=True)
df_club_games = spark.read.csv(path+"/archive/club_games.csv", header=True, inferSchema=True)
df_clubs = spark.read.csv(path+"/archive/clubs.csv", header=True, inferSchema=True)
df_competitions = spark.read.csv(path+"/archive/competitions.csv", header=True, inferSchema=True)
df_game_events = spark.read.csv(path+"/archive/game_events.csv", header=True, inferSchema=True)
df_games = spark.read.csv(path+"/archive/games.csv", header=True, inferSchema=True)
df_player_valutations = spark.read.csv(path+"/archive/player_valuations.csv", header=True, inferSchema=True)
df_players = spark.read.csv(path+"/archive/players.csv", header=True, inferSchema=True)


In [9]:
#print number of rows for each dataset
print("Number of rows: ", df_appearances.count())
print("Number of rows: ", df_club_games.count())
print("Number of rows: ", df_clubs.count())
print("Number of rows: ", df_competitions.count())
print("Number of rows: ", df_game_events.count())
print("Number of rows: ", df_games.count())
print("Number of rows: ", df_player_valutations.count())
print("Number of rows: ", df_players.count())

df_appearances.printSchema()
df_game_events.printSchema()
df_games.printSchema()

df_appearances.show()
df_game_events.show()
df_games.show()


Number of rows:  1166215
Number of rows:  123096
Number of rows:  411
Number of rows:  43
Number of rows:  559853
Number of rows:  61548
Number of rows:  421564
Number of rows:  28503
root
 |-- appearance_id: string (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_current_club_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)

root
 |-- game_id: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- club_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- description: string (nullable = true)
 

1 - unire players e appearances
2 - aggiungere la media punti per partita dei club per cui ha giocato in quell'anno
3 - aggiungere l'ultimo valore di mercato ottenuto in anno (player_valuations)

PRENDERE IN CONSIDERAZIONE LA POSSIBILITà DI ELIMINARE TOTALMENTE QUELLO CHE è SUCCESSO NEL 2023.

In [10]:
# join players and appearances
df_players_appearances = df_players.join(df_appearances, ["player_id"], how='inner')

In [11]:
df_players_appearances.show()

+---------+--------------------+---------------+-----------------+----------------------+-----------------+-------------+-------------+--------+--------------+-----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+------------+--------------------+--------------------+-----------+--------------------+-------------+-------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|                name|current_club_id|current_club_name|country_of_citizenship| country_of_birth|city_of_birth|date_of_birth|position|  sub_position| foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|   last_name|         player_code|           image_url|last_season|                 url|appearance_id|game_id|player_club_id|player_

In [12]:
#aggiungere a df_players_appearances la colonna year che contiene solo l'anno della feature date
df_players_appearances = df_players_appearances.withColumn("year", year(df_players_appearances.date))
#aggiungere a df_players_appearances la colonna month che contiene solo il mese della feature date
df_players_appearances = df_players_appearances.withColumn("month", month(df_players_appearances.date))


In [13]:
df_players_appearances.filter(df_players_appearances.player_id == 148455).show()

+---------+-------------+---------------+-----------------+----------------------+----------------+-----------------+-------------+--------+------------+----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+-------------+-------------+--------------------+-----------+--------------------+--------------+-------+--------------+----------------------+----------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+
|player_id|         name|current_club_id|current_club_name|country_of_citizenship|country_of_birth|    city_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|    last_name|  player_code|           image_url|last_season|                 url| appearance_id|game_id|player_club_id|player_current_club_id|    

In [14]:
# drop da df_players_appearances le colonne current_club_id, current_club_name, city_of_birth, market_value_in_eur, contract_expiration_date, agent_name, current_club_domestic_competition_id, image_url, last_season, url, game_id, player_current_club_id
df_players_appearances = df_players_appearances.drop("current_club_id", "date", "appearance_id", "highest_market_value_in_eur", "current_club_name", "city_of_birth", "market_value_in_eur", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "game_id", "player_current_club_id")

In [15]:
df_players_appearances.filter(df_players_appearances.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|first_name|    last_name|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Attack|Right Winger|Left|         175|      null|Mohamed Salah|mohamed-salah|           631|Mohamed Salah|            CL|

In [50]:
#TEST
print(df_players_appearances.filter(df_players_appearances.player_id == 148455).count())
df_players_appearances.filter(df_players_appearances.player_id == 148455).agg(sum(col("goals"))).show()

414
+--------------+
|sum(red_cards)|
+--------------+
|             0|
+--------------+



In [17]:
#aggiungere a df_players_appearances la colonna year che contiene solo l'anno della feature date
df_player_valutations = df_player_valutations.withColumn("year_v", year(df_player_valutations.date))
#aggiungere a df_players_appearances la colonna month che contiene solo il mese della feature date
df_player_valutations = df_player_valutations.withColumn("month_v", month(df_player_valutations.date))


In [18]:
df_player_valutations.show()

+----------+----------+----------+---------+---------------+-------------------+-----------------------------------+------+-------+
|      date|  datetime|  dateweek|player_id|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|
+----------+----------+----------+---------+---------------+-------------------+-----------------------------------+------+-------+
|2013-08-07|2013-08-07|2013-08-05|    99946|           1095|             150000|                                FR1|  2013|      8|
|2014-01-13|2014-01-13|2014-01-13|    99946|           1095|             100000|                                FR1|  2014|      1|
|2010-01-29|2010-01-29|2010-01-25|    76948|            979|             125000|                                PO1|  2010|      1|
|2010-08-20|2010-08-20|2010-08-16|    76948|            979|             250000|                                PO1|  2010|      8|
|2011-01-17|2011-01-17|2011-01-17|    76948|            979|             350

In [19]:
#TABLE valutation: per ogni valutazione abbiamo la coppia (mese, anno) e vogliamo estrarre dalla TABLE presenze la somma delle presenze comprese tra (mese, anno-1 e mese,anno)
# month, year di appearances
# month_v, year_v di valutations
from pyspark.sql.functions import year, month

#inserire una colonna "prev_year" nella tabella player_appearances che contiene l'anno precedente a quello della colonna year
#df_players_appearances = df_players_appearances.withColumn("prev_year", df_players_appearances.year - 1)

# Unisci A con B utilizzando la chiave (player_id, mese_anno)
AB = df_players_appearances.join(df_player_valutations, ["player_id"], "inner")

AB.filter(AB.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+----------+----------+----------+---------------+-------------------+-----------------------------------+------+-------+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|first_name|    last_name|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|      date|  datetime|  dateweek|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+---

In [20]:
#Filtriamo le istanze tenendo solo quelle che rispettano i vincoli sulle date
df_player_valutations_filtered = AB.filter(((AB["year"] == AB["year_v"]-1) & (AB["month"] >= AB["month_v"])) | ((AB["year"] == AB["year_v"]) & (AB["month"] <= AB["month_v"])))
#mostra
df_player_valutations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+----------+----------+----------+---------------+-------------------+-----------------------------------+------+-------+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|first_name|    last_name|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|      date|  datetime|  dateweek|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+---

'\n# Aggrega B raggruppando per la chiave (player_id, mese_anno) e calcolando la somma delle colonne a, b e c\nB_sum = B_filtered.groupBy("player_id", "mese_anno").agg({"a": "sum", "b": "sum", "c": "sum"})\n\n# Unisci il risultato dell\'aggregazione con A utilizzando la chiave (player_id, mese_anno)\nA_with_sum = A.join(B_sum, ["player_id", "mese_anno"], "left_outer")\n\n# Crea una nuova colonna "somma_abc" in A utilizzando la somma delle colonne a, b e c di B\nA_with_sum = A_with_sum.withColumn("somma_abc", A_with_sum["sum(a)"] + A_with_sum["sum(b)"] + A_with_sum["sum(c)"])\n'

In [ ]:
################# RIPRENDERE DA QUI #######################

In [22]:
# drop da df_player_valutations_filtered le colonne date, datetime, current_club_id, player_club_domestic_competition_id, current_club_id, player_club_domestic_competition_id
df_player_valutations_filtered = df_player_valutations_filtered.drop("date", "datetime", "current_club_id", "player_club_domestic_competition_id", "current_club_id", "player_club_domestic_competition_id")

In [23]:
df_player_valutations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+----------+-------------------+------+-------+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|first_name|    last_name|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|  dateweek|market_value_in_eur|year_v|month_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+----------+-------------------+------+-------+
|   148455|Mohamed Salah|                 Egypt|           Egypt

In [25]:
df_player_valutations_filtered.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- player_code: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dateweek: date (nullable = true)
 |-- market_value_in

In [37]:
from pyspark.sql.functions import sum, col

df_new = df_player_valutations_filtered.groupBy("player_id", "competition_id", "player_club_id", "year_v", "month_v", "market_value_in_eur").agg(
    sum(col("yellow_cards")).alias("yellow_cards"),
    sum(col("red_cards")).alias("red_cards"),
    sum(col("goals")).alias("goals"),
    sum(col("assists")).alias("assists"),
    sum(col("minutes_played")).alias("minutes_played")
).select("player_id", "competition_id", "player_club_id", "month_v", "year_v", "yellow_cards", "red_cards", "goals", "assists", "minutes_played", "market_value_in_eur").distinct()

In [34]:
#df_player_valutations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()
df_new.filter(df_new.player_id == 148455).show(100)

+---------+--------------+--------------+-------+------+------------+---------+-----+-------+--------------+-------------------+
|player_id|competition_id|player_club_id|month_v|year_v|yellow_cards|red_cards|goals|assists|minutes_played|market_value_in_eur|
+---------+--------------+--------------+-------+------+------------+---------+-----+-------+--------------+-------------------+
|   148455|            CL|           631|      2|  2015|           0|        0|    0|      0|            77|           10000000|
|   148455|            CL|           631|      3|  2015|           0|        0|    0|      0|            77|           12000000|
|   148455|            CL|           631|      7|  2015|           0|        0|    0|      0|            77|           18000000|
|   148455|            CL|           631|     11|  2015|           0|        0|    0|      0|            71|           20000000|
|   148455|            EL|           430|      2|  2015|           1|        0|    1|      0|    

In [35]:
#TEST
print(df_new.filter(df_new.player_id == 148455).count())
df_new.filter(df_new.player_id == 148455).agg(sum(col("goals"))).show()

114
+----------+
|sum(goals)|
+----------+
|       574|
+----------+



In [38]:
from pyspark.sql.functions import collect_set, sum

# Raggruppare per (player_id, year) e raccogliere i valori unici di competition_id e player_club_id in liste
df_grouped = df_new.groupBy("player_id", "year_v", "month_v", "market_value_in_eur") \
              .agg(collect_set("competition_id").alias("competition_id_list"),
                   collect_set("player_club_id").alias("player_club_id_list"))

# Unire il dataframe originale con quello ottenuto al passaggio precedente sulla chiave (player_id, year)
# e calcolare la somma delle colonne yellow_cards, red_cards, goals, assists e minutes_played raggruppando sulla nuova chiave (player_id, competition_id, year, player_club_id)
df_result = df_new.join(df_grouped, ["player_id", "year_v", "month_v", "market_value_in_eur"]) \
              .groupBy("player_id", "competition_id", "month_v", "year_v", "player_club_id", "market_value_in_eur") \
              .agg(sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played")) \
              .groupBy("player_id", "year_v", "month_v", "market_value_in_eur") \
              .agg(collect_set("competition_id").alias("competition_id"),
                   collect_set("player_club_id").alias("player_club_id"),
                   sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played"))

# Visualizza il risultato
df_result.show()

+---------+------+-------+-------------------+------------------+----------------+------------+---------+-----+-------+--------------+
|player_id|year_v|month_v|market_value_in_eur|    competition_id|  player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+------+-------+-------------------+------------------+----------------+------------+---------+-----+-------+--------------+
|       26|  2015|      2|            3000000|          [CL, L1]|            [16]|           0|        0|    0|      0|          2160|
|       26|  2015|      7|            2000000|          [CL, L1]|            [16]|           0|        0|    0|      0|          2880|
|       26|  2015|     10|            1000000| [CL, ELQ, L1, EL]|            [16]|           0|        0|    0|      0|          2880|
|       26|  2016|      2|            1000000| [CL, ELQ, L1, EL]|            [16]|           0|        0|    0|      0|          2070|
|       26|  2016|      7|            1000000|     [ELQ

In [39]:
#df_new.filter(df_new.player_id == 148455).show(100)
df_result.filter(df_result.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|   148455|  2015|      2|           10000000|[CL, FAC, EL, IT1...|    [430, 631]|           1|        0|    3|      2|           488|
|   148455|  2015|      3|           12000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    6|      3|           900|
|   148455|  2015|      7|           18000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    9|      5|          1970|
|   148455|  2015|     11|           20000000|[CL, FAC, CIT, EL...|[12, 430, 631]|           3|        0|   15|      5|          3113|
|   148455|  2016|      1|           20000000|[CL, FAC,

In [40]:
#TEST (a questo punto il test non ha piu valore dal momento che si tiene conto per ogni valutazione quello che si è fatto l'anno precedente a year_v--month_v)
print(df_result.filter(df_result.player_id == 148455).count())
df_result.filter(df_result.player_id == 148455).agg(sum(col("goals"))).show()

23
+----------+
|sum(goals)|
+----------+
|       574|
+----------+



In [43]:
#unire df_result con df_players ma di df_players non voglio la colonna "market_value_in_eur"
df_players = df_players.drop("market_value_in_eur")
df_stats = df_result.join(df_players, ["player_id"], "inner")
df_stats.filter(df_stats.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+---------------+-----------------+----------------------+----------------+-----------------+-------------+--------+------------+----+------------+---------------------------+----------+------------------------+------------------------------------+----------+-------------+-------------+--------------------+-----------+--------------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|current_club_id|current_club_name|country_of_citizenship|country_of_birth|    city_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|    last_name|  player_code|           image_url|last_season|                 url|
+---------+------+----

In [44]:
df_stats = df_stats.drop("current_club_id", "current_club_name", "city_of_birth", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "player_current_club_id")

In [47]:
df_stats = df_stats.drop("highest_market_value", "player_code")

In [48]:
df_stats.filter(df_stats.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|first_name|    last_name|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+
|   148455|  2015|      2|           10000000|[CL, FAC, EL, IT1...|    [430, 631]|           1|        0|    3| 

In [49]:
print("Number of rows: ", df_stats.count())

Number of rows:  141441


In [1]:
# a df_stats aggiungere la colonna età (semplicemnte data di nascita - (year_v-month_v) ---> putroppo abbiamo perso il giorno della data della valutazione, per recuperarla si deve evitare di
# perdere o scartare la colonna dateweek che è ancora presente nella tabella data in output nella cella 21 of 34 df_player_valutations_filtered)

# a df_stats aggiungere la colonna partite_giocate che contiene il numero di partite giocate dalle sue squadre (player_club_id) in year;

# a df_stats aggiungere la colonna punti_ottenuti che contiene il numero di punti ottenuti dalle sue squadre (player_club_id) in year; (3 punti per vittoria, 1 per pareggio, 0 per sconfitta)

# a df_stats aggiungere la colonna media_punti che contiene la media dei punti ottenuti dalle sue squadre (player_club_id) in year; (ottenerla dalle colonne partite_giocate e punti_ottenuti)